In [24]:
import pandas as pd
from PIL import Image,ImageDraw, ImageFont,ImageFilter
import re
import random
import colorsys

In [25]:
df = pd.read_csv('samples.csv')

In [26]:
df

,string
0,ぁ
1,あ
2,ぃ
3,い
4,ぅ
...,...
46255,脇
46256,惑
46257,枠
46258,湾


In [27]:
def generate_random_hiragana(n):
    hiragana_pattern = re.compile('[\u3041-\u3096]')
    return ''.join([random.choice(hiragana_pattern.findall(chr(random.randint(0x3041, 0x3096)))) for _ in range(n)])

In [28]:

def generate_random_color():
    # Generate random RGB values excluding black
    red = random.randint(20, 255)
    green = random.randint(20, 255)
    blue = random.randint(20, 255)

    # Format the RGB values into a tuple
    color_tuple = (red, green, blue)

    return color_tuple


In [29]:
text = df['string']

In [30]:
from PIL import Image, ImageDraw, ImageFont, ImageFilter
import random

def generate_img_with_random_furi(sentence, file_name):
    # Specify the font and size
    font_path = "fonts/NotoSerifJP/NotoSerifJP-Black.otf"  # Replace with the path to your TTF font file
    font_size = 25
    font = ImageFont.truetype(font_path, font_size)

    furigana_font_size = font_size // 2
    furigana_font = ImageFont.truetype(font_path, furigana_font_size)

    # Specify the image size
    image_size = (50,50)

    # Specify the initial position for the text
    initial_position = ((image_size[0] - font_size) // 2, (image_size[1] - len(sentence) * font_size) // 2 - 15)

    # Create the image
    color = "white"
    if random.random()<0.2:
        color = generate_random_color()
    image = Image.new("RGB", image_size, color)

    # Create a draw object
    draw = ImageDraw.Draw(image)

    for char in sentence:
        draw.text(initial_position, char, font=font, fill="black")
        # random furigana
        if random.random() < 0.2:
            furigana_txt = generate_random_hiragana(2)
            furigana_x, furigana_y = initial_position
            furigana_x += font_size
            furigana_y += 5
            for furigana_char in furigana_txt:
                draw.text((furigana_x, furigana_y), furigana_char, font=furigana_font, fill='black')
                furigana_y += furigana_font_size
        initial_position = (initial_position[0], initial_position[1] + font_size)  # Move down for each character

    # randomly blur
    if random.random() < 0.3:
        # add blur
        mean_radius = 0.5
        std_dev_radius = 0.2
        blur_radius = max(0, min(1, random.gauss(mean_radius, std_dev_radius)))
        image = image.filter(ImageFilter.GaussianBlur(radius=blur_radius))

    # randomly shear
    if random.random() < 0.3:
        mean_shear = 0.0
        std_dev_shear = 0.1
        shear_amount = random.gauss(mean_shear, std_dev_shear)
        image = image.transform(
            image_size,
            Image.AFFINE,
            (1, shear_amount, 0, 0, 1, 0),
            resample=Image.BICUBIC
        )

    # Save or display the image
    image.save(file_name)

# Your other functions (e.g., generate_random_color, generate_random_hiragana) should be defined before calling this function.


In [31]:
generate_img_with_random_furi(text.iloc[0],'out.jpg')

In [33]:
counter = 0
# the file name is just the index of the label
dict_training = {'labels':[]}

limit_broken = False
for sentence in text:
    file_name = f'{counter}.jpg'
    generate_img_with_random_furi(sentence,f'./train_imgs/{file_name}')
    dict_training['labels'].append(sentence)
    counter += 1


In [34]:
df = pd.DataFrame(dict_training)

# Specify the CSV file path
csv_file_path = 'training_labels.csv'

# Save the DataFrame to CSV
df.to_csv(csv_file_path, index=False)


In [ ]:
df

,labels
0,ぁ
1,あ
2,ぃ
3,い
4,ぅ
...,...
11560,脇
11561,惑
11562,枠
11563,湾
